In [1]:
import cobra

In [2]:
model = cobra.io.load_json_model('../GEMs/iRsp1140_opt.json');

In [3]:
model.optimize()

,fluxes,reduced_costs
RXN0001_c0,-298.604625,-9.926868e-19
RXN0002_c0,0.000000,-1.111147e-03
RXN0003_c0,-1000.000000,-1.206389e-02
RXN0005_c0,4.195156,0.000000e+00
RXN0006_c0,0.000000,-4.656237e-03
...,...,...
EX_cpd11416_c0,122.058656,0.000000e+00
Biomass_Aero,0.000000,-8.987584e-01
Biomass_Photo,0.000000,-2.000000e+00
RXN1826_c0,1.360710,1.194025e-17


In [4]:
#familiarize with model summary
print(len(model.reactions))

1647


# OPTIMIZING MODEL


In [32]:
from cobra.io import load_model
from cobra.flux_analysis import gapfill
model = cobra.io.read_sbml_model('../GEMs/iRsp1140_opt_compartment_updated.xml')
model.optimize()

,fluxes,reduced_costs
RXN0001_c0,-298.604625,9.926868e-19
RXN0002_c0,0.000000,-1.111147e-03
RXN0003_c0,-1000.000000,-1.206389e-02
RXN0005_c0,4.195156,0.000000e+00
RXN0006_c0,0.000000,-4.656237e-03
...,...,...
EX_cpd11416_c0,122.058656,0.000000e+00
Biomass_Aero,0.000000,-8.987584e-01
Biomass_Photo,0.000000,-2.000000e+00
RXN1826_c0,1.360710,1.102177e-17


In [34]:
model.metabolites.f6p_c.reactions

frozenset({<Reaction RXN0046_c0 at 0x2b3199e1520>,
           <Reaction RXN0343_c0 at 0x2b319aabda0>,
           <Reaction RXN0345_c0 at 0x2b319aabdd0>,
           <Reaction RXN0415_c0 at 0x2b319b1cb90>,
           <Reaction RXN0416_c0 at 0x2b319add490>,
           <Reaction RXN0553_c0 at 0x2b319b9f620>,
           <Reaction RXN0556_c0 at 0x2b319b9f980>,
           <Reaction RXN0560_c0 at 0x2b319b9fe60>})

In [35]:
import cobra
universal = cobra.Model("universal_reactions")
for i in [i.id for i in model.metabolites.f6p_c.reactions]:
    reaction = model.reactions.get_by_id(i)
    universal.add_reactions([reaction.copy()])
    model.remove_reactions([reaction])

In [36]:
model.optimize().objective_value

0.0

In [37]:
solution = gapfill(model, universal, demand_reactions=False)
for reaction in solution[0]:
    print(reaction.id)

RXN0560_c0
RXN0345_c0


In [24]:
universal.reactions


[]

In [17]:
for metabolite in model.metabolites:
    note = metabolite.notes
    if type(note) is str:
        metabolite.notes = {'seed.id': note}

In [18]:
cobra.io.save_json_model(model, "../GEMs/iRsp1140_opt.json")
cobra.io.write_sbml_model(model, "../rhodobacter/model_gapfilled.xml")

# reverse glyoxylate shunt (rGS)

ppc, added\
mdh: RXN0143_c0, reversible\
mtk, added\
fum: RXN0148_c0, reversible\
frd: RXN0145_c0, reversible\
mcl: RXN0689_c0, reversible\
icl, added\
acn: RXN0147_c0, reversible\
acl, added

In [4]:
def add_ppc_reaction(model):
    # H2O + Phosphoenolpyruvate + CO2 -> Orthophosphate + Oxaloacetate
    reaction = cobra.Reaction('ppc')
    reaction.name = 'phosphate:oxaloacetate carboxy-lyase'
    reaction.subsystem = 'reverse glyoxylate shunt'
    reaction.add_metabolites({
        model.metabolites.get_by_id('h2o_c0'): -1.0,
        model.metabolites.get_by_id('pep_c0'): -1.0,
        model.metabolites.get_by_id('co2_c0'): -1.0, 
        model.metabolites.get_by_id('pi_c0'): 1.0,
        model.metabolites.get_by_id('oaa_c0'): 1.0
    })
    model.add_reactions([reaction])
    return model

def add_mtk_reaction(model):
    # ATP + (S)-Malate + CoA -> ADP + Orthophosphate + L-Malyl-CoA
    # Note: existing reaction RXN0690_c0 can convert L-Malyl-CoA to (S)-Malate + CoA
    reaction = cobra.Reaction('mtk')
    reaction.name = 'malate thiokinase'
    reaction.subsystem = 'reverse glyoxylate shunt'
    reaction.add_metabolites({
        model.metabolites.get_by_id('atp_c0'): -1.0,
        model.metabolites.get_by_id('mal-L_c0'): -1.0,
        model.metabolites.get_by_id('coa_c0'): -1.0,
        model.metabolites.get_by_id('adp_c0'): 1.0,
        model.metabolites.get_by_id('pi_c0'): 1.0,
        model.metabolites.get_by_id('malylcoa_c0'): 1.0
    })
    model.add_reactions([reaction])
    return model

def add_icl_reaction(model):
    # succinate + glyoxylate <=> isocitrate (reversible)
    reaction = cobra.Reaction('icl')
    reaction.name = 'isocitrate lyase' 
    reaction.subsystem = 'reverse glyoxylate shunt'
    reaction.add_metabolites({
        model.metabolites.get_by_id('glx_c0'): -1.0,
        model.metabolites.get_by_id('succ_c0'): -1.0,
        model.metabolites.get_by_id('icit_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_acl_reaction(model):
    # ATP + Citrate + CoA -> ADP + Orthophosphate + Acetyl-CoA + Oxaloacetate
    reaction = cobra.Reaction('acl')
    reaction.name = 'acetyl-CoA:oxaloacetate C-acetyltransferase'
    reaction.subsystem = 'reverse glyoxylate shunt'
    reaction.add_metabolites({
        model.metabolites.get_by_id('atp_c0'): -1.0,
        model.metabolites.get_by_id('cit_c0'): -1.0,
        model.metabolites.get_by_id('coa_c0'): -1.0,
        model.metabolites.get_by_id('adp_c0'): 1.0,
        model.metabolites.get_by_id('pi_c0'): 1.0,
        model.metabolites.get_by_id('accoa_c0'): 1.0,
        model.metabolites.get_by_id('oaa_c0'): 1.0
    })
    model.add_reactions([reaction])
    return model

def add_rGS_reactions(model):
    model = add_ppc_reaction(model)
    model = add_mtk_reaction(model)
    model = add_icl_reaction(model)
    model = add_acl_reaction(model)
    return model

In [5]:
model_rGS = model.copy()
model_rGS = add_rGS_reactions(model_rGS)

In [6]:
model_rGS

Name,Rhodobacter_sphaeroides_2_4_1_fbamdl_17
Memory address,3064e5fc0
Number of metabolites,1344
Number of reactions,1651
Number of genes,1140
Number of groups,0
Objective expression,1.0*biomass0 - 1.0*biomass0_reverse_9c18d
Compartments,"Cytosol_0, e0, p0"


In [8]:
cobra.io.save_json_model(model_rGS, "../GEMs/iRsp1140_opt_with_rGS.json")

# non-oxidative glycolysis (NOG)

fpk, added\
xpk, added\
tkt, RXN0553_c0\
tal, RXN0560_c0\
rpi, RXN0561_c0\
rpe, RXN0559_c0

In [9]:
def add_NOG_reactions(model):
    # D-Xylulose 5-phosphate + Orthophosphate <=> Acetyl phosphate + D-Glyceraldehyde 3-phosphate + H2O
    reaction = cobra.Reaction('xpk')
    reaction.name = 'D-xylulose 5-phosphate D-glyceraldehyde-3-phosphate-lyase'
    reaction.subsystem = 'non-oxidative glycolysis'
    reaction.add_metabolites({
        model.metabolites.get_by_id('xu5p-D_c0'): -1.0,
        model.metabolites.get_by_id('pi_c0'): -1.0,
        model.metabolites.get_by_id('actp_c0'): 1.0,
        model.metabolites.get_by_id('g3p_c0'): 1.0,
        model.metabolites.get_by_id('h2o_c0'): 1.0
    })
    model.add_reactions([reaction])

    # D-Fructose 6-phosphate + Orthophosphate <=> Acetyl phosphate + D-Erythrose 4-phosphate + H2O
    reaction = cobra.Reaction('fpk')
    reaction.name = 'D-fructose-6-phosphate D-erythrose-4-phosphate-lyase'
    reaction.subsystem = 'non-oxidative glycolysis'
    reaction.add_metabolites({
        model.metabolites.get_by_id('f6p_c0'): -1.0,
        model.metabolites.get_by_id('pi_c0'): -1.0,
        model.metabolites.get_by_id('actp_c0'): 1.0,
        model.metabolites.get_by_id('e4p_c0'): 1.0,
        model.metabolites.get_by_id('h2o_c0'): 1.0
    })
    model.add_reactions([reaction])
    return model

In [10]:
model_NOG = model.copy()
model_NOG = add_NOG_reactions(model_NOG)

In [11]:
model_NOG

Name,Rhodobacter_sphaeroides_2_4_1_fbamdl_17
Memory address,147351030
Number of metabolites,1344
Number of reactions,1649
Number of genes,1140
Number of groups,0
Objective expression,1.0*biomass0 - 1.0*biomass0_reverse_9c18d
Compartments,"Cytosol_0, e0, p0"


In [12]:
cobra.io.save_json_model(model_NOG, "../GEMs/iRsp1140_opt_with_NOG.json")

# Embden-Parnas-Bifido (EP-Bifido) pathway

In [13]:
def add_gnd_reaction(model):
    # 6-Phospho-D-gluconate + NADP+ <=> D-Ribulose 5-phosphate + CO2 + NADPH + H+
    reaction = cobra.Reaction('gnd')
    reaction.name = '6-phospho-D-gluconate:NADP+ 2-oxidoreductase'
    reaction.subsystem = 'EP-Bifido pathway'
    reaction.add_metabolites({
        model.metabolites.get_by_id('6pgc_c0'): -1.0,
        model.metabolites.get_by_id('nadp_c0'): -1.0,
        model.metabolites.get_by_id('ru5p-D_c0'): 1.0,
        model.metabolites.get_by_id('co2_c0'): 1.0,
        model.metabolites.get_by_id('nadph_c0'): 1.0,
        model.metabolites.get_by_id('h_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_EP_Bifido_reactions(model):
    model = add_NOG_reactions(model)
    model = add_gnd_reaction(model)
    return model

In [14]:
model_EP_Bifido = model.copy()
model_EP_Bifido = add_EP_Bifido_reactions(model_EP_Bifido)

In [15]:
model_EP_Bifido

Name,Rhodobacter_sphaeroides_2_4_1_fbamdl_17
Memory address,3083bcbb0
Number of metabolites,1344
Number of reactions,1650
Number of genes,1140
Number of groups,0
Objective expression,1.0*biomass0 - 1.0*biomass0_reverse_9c18d
Compartments,"Cytosol_0, e0, p0"


In [16]:
cobra.io.save_json_model(model_EP_Bifido, "../GEMs/iRsp1140_opt_with_EP_Bifido.json")

# malyl-CoA-glycerate cylce (MCG)

ppc, added\
mdh: RXN0143_c0, reversible\
mtk, added\
mcl: RXN0689_c0, reversible\
gcl, added\
tsr, added\
gk, added\
eno: RXN0420_c0

In [17]:
def add_2h3oppan_c0_metabolite(model):
    m_2h3oppan_c0 = cobra.Metabolite('2h3oppan_c0', formula='C3H3O4', name='2-Hydroxy-3-oxopropanoate', compartment='c0', charge = -1) #'cpd00843'
    model.add_metabolites([m_2h3oppan_c0])
    return model

def add_gcl_reaction(model):
    # 2 Glyoxylate -> 2-Hydroxy-3-oxopropanoate + CO2
    reaction = cobra.Reaction('gcl')
    reaction.name = 'glyoxylate carboxy-lyase'
    reaction.subsystem = 'malyl-CoA-glycerate cylce'
    reaction.add_metabolites({
        model.metabolites.get_by_id('glx_c0'): -2.0,
        model.metabolites.get_by_id('2h3oppan_c0'): 1.0,
        model.metabolites.get_by_id('co2_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_tsr_reaction(model):
    # 2-Hydroxy-3-oxopropanoate + NADH + H+ -> D-Glycerate + NAD+
    reaction = cobra.Reaction('tsr')
    reaction.name = '(R)-glycerate:NAD+ oxidoreductase'
    reaction.subsystem = 'malyl-CoA-glycerate cylce'
    reaction.add_metabolites({
        model.metabolites.get_by_id('2h3oppan_c0'): -1.0,
        model.metabolites.get_by_id('nadh_c0'): -1.0,
        model.metabolites.get_by_id('h_c0'): -1.0,
        model.metabolites.get_by_id('glyc-R_c0'): 1.0,
        model.metabolites.get_by_id('nad_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_gk_reaction(model):
    # D-Glycerate + ATP -> 2-Phospho-D-glycerate + ADP
    reaction = cobra.Reaction('gk')
    reaction.name = 'ATP:(R)-glycerate 2-phosphotransferase'
    reaction.subsystem = 'EP-Bifido pathway'
    reaction.add_metabolites({
        model.metabolites.get_by_id('glyc-R_c0'): -1.0,
        model.metabolites.get_by_id('atp_c0'): -1.0,
        model.metabolites.get_by_id('2pg_c0'): 1.0,
        model.metabolites.get_by_id('adp_c0'): 1.0,
    })
    model.add_reactions([reaction])
    return model

def add_MCG_reactions(model):
    model = add_ppc_reaction(model)
    model = add_mtk_reaction(model)
    model = add_2h3oppan_c0_metabolite(model)
    model = add_gcl_reaction(model)
    model = add_tsr_reaction(model)
    model = add_gk_reaction(model)
    return model

In [18]:
model_MCG = model.copy()
model_MCG = add_MCG_reactions(model_MCG)

In [19]:
model_MCG

Name,Rhodobacter_sphaeroides_2_4_1_fbamdl_17
Memory address,308db4850
Number of metabolites,1345
Number of reactions,1652
Number of genes,1140
Number of groups,0
Objective expression,1.0*biomass0 - 1.0*biomass0_reverse_9c18d
Compartments,"Cytosol_0, e0, p0"


In [20]:
cobra.io.save_json_model(model_MCG, "../GEMs/iRsp1140_opt_with_MCG.json")